In [2]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import TensorBoard
from keras import backend as K


In [3]:
batch_size = 10
num_classes = 10
epochs = 50

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
def make_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='sigmoid'))

    model.compile(loss=keras.losses.binary_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['binary_accuracy', 'categorical_accuracy', 'mse'])
    return model

In [6]:
logdir = "~/data/log/mnist/sigmoid"
!mkdir -p $logdir
K.clear_session()
model = make_model()
model.fit(x_train[::60], y_train[::60],
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test[::10], y_test[::10]),
         callbacks=[
             TensorBoard(
                 logdir, histogram_freq=1, write_grads=True, batch_size=10
             )
         ])


Train on 1000 samples, validate on 1000 samples
Epoch 1/50
1000/1000 [==============================] - 2s 2ms/step - loss: 0.2748 - binary_accuracy: 0.9048 - categorical_accuracy: 0.4510 - mean_squared_error: 0.0788 - val_loss: 0.1551 - val_binary_accuracy: 0.9462 - val_categorical_accuracy: 0.7320 - val_mean_squared_error: 0.0428
Epoch 2/50
1000/1000 [==============================] - 0s 408us/step - loss: 0.1671 - binary_accuracy: 0.9391 - categorical_accuracy: 0.7030 - mean_squared_error: 0.0464 - val_loss: 0.1173 - val_binary_accuracy: 0.9564 - val_categorical_accuracy: 0.7960 - val_mean_squared_error: 0.0329
Epoch 3/50
1000/1000 [==============================] - 0s 411us/step - loss: 0.1263 - binary_accuracy: 0.9531 - categorical_accuracy: 0.7970 - mean_squared_error: 0.0355 - val_loss: 0.0846 - val_binary_accuracy: 0.9719 - val_categorical_accuracy: 0.8640 - val_mean_squared_error: 0.0226
Epoch 4/50
1000/1000 [==============================] - 0s 412us/step - loss: 0.1027 - bin

1000/1000 [==============================] - 0s 406us/step - loss: 0.0078 - binary_accuracy: 0.9971 - categorical_accuracy: 0.9940 - mean_squared_error: 0.0021 - val_loss: 0.0444 - val_binary_accuracy: 0.9876 - val_categorical_accuracy: 0.9410 - val_mean_squared_error: 0.0102
Epoch 30/50
1000/1000 [==============================] - 0s 411us/step - loss: 0.0091 - binary_accuracy: 0.9962 - categorical_accuracy: 0.9940 - mean_squared_error: 0.0025 - val_loss: 0.0436 - val_binary_accuracy: 0.9879 - val_categorical_accuracy: 0.9490 - val_mean_squared_error: 0.0101
Epoch 31/50
 780/1000 [======================>.......] - ETA: 0s - loss: 0.0107 - binary_accuracy: 0.9960 - categorical_accuracy: 0.9910 - mean_squared_error: 0.0030

KeyboardInterrupt: 

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])